In [1]:
import numpy as np
import pandas as pd
from utils.load_data import fma_load

features = fma_load('data/fma_metadata/features.csv')

features

tracks = fma_load('data/fma_metadata/tracks.csv')

y = tracks['track','genre_top']

corr = features.corr()

threshold = 0.75

corr_matrix = corr.copy()

np.fill_diagonal(corr_matrix.values, 0) # Diagonal to zero

strong_corr = (corr_matrix.abs() > threshold)

pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if strong_corr.iloc[i, j]:
            pairs.append({
                "feature_1": corr_matrix.columns[i],
                "feature_2": corr_matrix.columns[j],
                "correlation": corr_matrix.iloc[i, j]
            })
pairs



[{'feature_1': ('chroma_cens', 'mean', '01'),
  'feature_2': ('chroma_cens', 'median', '01'),
  'correlation': np.float64(0.9717560508855922)},
 {'feature_1': ('chroma_cens', 'mean', '01'),
  'feature_2': ('chroma_cqt', 'mean', '01'),
  'correlation': np.float64(0.9281310171436797)},
 {'feature_1': ('chroma_cens', 'mean', '01'),
  'feature_2': ('chroma_cqt', 'median', '01'),
  'correlation': np.float64(0.9147936958742171)},
 {'feature_1': ('chroma_cens', 'mean', '02'),
  'feature_2': ('chroma_cens', 'median', '02'),
  'correlation': np.float64(0.9710105562557106)},
 {'feature_1': ('chroma_cens', 'mean', '02'),
  'feature_2': ('chroma_cqt', 'mean', '02'),
  'correlation': np.float64(0.8924722909612783)},
 {'feature_1': ('chroma_cens', 'mean', '02'),
  'feature_2': ('chroma_cqt', 'median', '02'),
  'correlation': np.float64(0.891592107193247)},
 {'feature_1': ('chroma_cens', 'mean', '02'),
  'feature_2': ('chroma_cqt', 'skew', '02'),
  'correlation': np.float64(-0.7727008716214067)},
 {'

In [5]:
%pip install -r requirements.txt

  Using cached scikit_learn-1.8.0-cp313-cp313-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (11 kB)
  Using cached matplotlib-3.10.8-cp313-cp313-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (52 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scipy-1.16.3-cp313-cp313-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached contourpy-1.3.3-cp313-cp313-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.1-cp313-cp313-manylinux1_x86_64.manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_5_x86_64.whl.metadata (114 kB)
  Using cached kiwisolver-1.4.9-cp313-cp313-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (6.3 kB)
  Using cached pyparsing-3.3.1-py3-none-any.whl.m

In [2]:
features.drop(columns=['chroma_cqt','chroma_cens'],inplace=True)
features = features.loc[:, features.columns.get_level_values('statistics') != 'median']

features

feature    chroma_stft                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2            -1.006041 -0.634076 -0.233752 -0.120917  0.004806  1.218982   
3            -0.951502 -0.660734 -1.050015 -0.977441 -0.343043 -0.515404   
5            -0.794551 -1.264806 -0.664387 -0.405196 -0.022688  0.014883   
10            0.954931 -1.266404  0.030425 -0.646823  0.664217  0.082626   
20           -0.048443 -0.543755 -1.336000 -0.410307 -0.684526 -0.960676   
...                ...       ...       ...       ...       ...       ...   
155316       -0.959207 -0.130793 -0.635587 -0.251981 -0.656534 -0.238574   
155317       -0.965068 -0.679953 -1.051353 -0.852524 -0.872679 -0.740262   
155318       -0.771414 -0.438261 -0.888968 -0.730804 -0.868208 -0.623147   
155319       -0.984837 -0.198621 -0.923624 -0.825457 -1.035743 -0.919991   
155320       -0.787214 -0.792066 -1.052318 -0.930889 -1.111142 -0.961365   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        03        04   
track_id                                            ...                       
2           0.969103 -0.884986 -0.469190 -0.515475  ...  0.038974  0.054125   
3          -0.973297 -1.261086 -1.132458 -0.953374  ...  0.051151  0.063831   
5          -0.190766 -0.507027 -0.868905 -0.952605  ...  0.084997  0.040730   
10         -1.241245  0.102074 -0.742071  0.670849  ...  0.088197  0.074358   
20         -1.108473 -1.320151 -0.754329 -1.029887  ...  0.105521  0.095003   
...              ...       ...       ...       ...  ...       ...       ...   
155316     -0.372219 -0.661802 -0.297405 -0.533092  ...  0.102859  0.128410   
155317     -0.737960 -1.060661 -1.029553 -0.949072  ...  0.135479  0.132964   
155318     -0.855489 -0.950127 -0.725640 -0.694402  ...  0.089910  0.108324   
155319     -0.655853 -0.887433 -0.398968 -0.233844  ...  0.092314  0.088311   
155320     -0.890171 -0.987430 -0.638198 -1.095618  ...  0.099553  0.091421   

feature                               zcr                                \
statistics                       kurtosis       max      mean       min   
number            05        06         01        01        01        01   
track_id                                                                  
2           0.012226  0.012111   5.758890  0.459473  0.085629  0.000000   
3           0.014212  0.017740   2.824694  0.466309  0.084578  0.000000   
5           0.012691  0.014759   6.808415  0.375000  0.053114  0.000000   
10          0.017952  0.013921  21.434212  0.452148  0.077515  0.000000   
20          0.022492  0.021355  16.669037  0.469727  0.047225  0.000977   
...              ...       ...        ...       ...       ...       ...   
155316      0.022547  0.019816   4.448255  0.172852  0.028773  0.003906   
155317      0.023548  0.026527   3.270612  0.196289  0.031116  0.002441   
155318      0.017540  0.020471   2.356727  0.212891  0.038450  0.003418   
155319      0.018328  0.017936   6.188604  0.167480  0.041480  0.004883   
155320      0.020312  0.016794  21.756050  0.845215  0.075141  0.004395   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
3           1.716724  0.069330  
5           2.193303  0.044861  
10          3.542325  0.040800  
20          3.189831  0.030993  
...              ...       ...  
155316      0.955388  0.012385  
155317      1.283060  0.019059  
155318      0.828569  0.017904  
155319      1.818740  0.020133  
155320      4.687204  0.137205  

[106574 rows x 300 columns]

In [24]:
from collections import Counter

df_flat = features.copy()
df_flat.columns = ['_'.join(col).strip() for col in df_flat.columns.values]

df_flat['genre_top'] = y

df_flat.dropna(inplace=True)

counts = Counter(df_flat['genre_top'])

rare_classes = [cls for cls, c in counts.items() if c < 400]

df_flat["genre_top_grouped"] = df_flat["genre_top"].replace(
    {cls: "Other" for cls in rare_classes}
)

df_flat

/tmp/ipykernel_3101/438632209.py:14: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_flat["genre_top_grouped"] = df_flat["genre_top"].replace(


,chroma_stft_kurtosis_01,chroma_stft_kurtosis_02,chroma_stft_kurtosis_03,chroma_stft_kurtosis_04,chroma_stft_kurtosis_05,chroma_stft_kurtosis_06,chroma_stft_kurtosis_07,chroma_stft_kurtosis_08,chroma_stft_kurtosis_09,chroma_stft_kurtosis_10,...,tonnetz_std_05,tonnetz_std_06,zcr_kurtosis_01,zcr_max_01,zcr_mean_01,zcr_min_01,zcr_skew_01,zcr_std_01,genre_top,genre_top_grouped
track_id,,,,,,,,,,,,,,,,,,,,,
2,-1.006041,-0.634076,-0.233752,-0.120917,0.004806,1.218982,0.969103,-0.884986,-0.469190,-0.515475,...,0.012226,0.012111,5.758890,0.459473,0.085629,0.000000,2.089872,0.061448,Hip-Hop,Hip-Hop
3,-0.951502,-0.660734,-1.050015,-0.977441,-0.343043,-0.515404,-0.973297,-1.261086,-1.132458,-0.953374,...,0.014212,0.017740,2.824694,0.466309,0.084578,0.000000,1.716724,0.069330,Hip-Hop,Hip-Hop
5,-0.794551,-1.264806,-0.664387,-0.405196,-0.022688,0.014883,-0.190766,-0.507027,-0.868905,-0.952605,...,0.012691,0.014759,6.808415,0.375000,0.053114,0.000000,2.193303,0.044861,Hip-Hop,Hip-Hop
10,0.954931,-1.266404,0.030425,-0.646823,0.664217,0.082626,-1.241245,0.102074,-0.742071,0.670849,...,0.017952,0.013921,21.434212,0.452148,0.077515,0.000000,3.542325,0.040800,Pop,Pop
134,-0.972450,-1.047674,-1.312579,-1.120184,-1.280836,-1.215888,-1.006120,-0.915774,-0.781248,-1.012533,...,0.016322,0.015819,4.731087,0.419434,0.064370,0.000000,1.806106,0.054623,Hip-Hop,Hip-Hop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155315,-0.931622,-0.823569,-1.235423,-1.213406,-0.999892,-0.672000,-1.219753,-1.381937,-1.270755,-1.129524,...,0.016631,0.014705,6.947788,0.188477,0.037708,0.002930,1.764233,0.018679,Rock,Rock
155316,-0.959207,-0.130793,-0.635587,-0.251981,-0.656534,-0.238574,-0.372219,-0.661802,-0.297405,-0.533092,...,0.022547,0.019816,4.448255,0.172852,0.028773,0.003906,0.955388,0.012385,Rock,Rock
155317,-0.965068,-0.679953,-1.051353,-0.852524,-0.872679,-0.740262,-0.737960,-1.060661,-1.029553,-0.949072,...,0.023548,0.026527,3.270612,0.196289,0.031116,0.002441,1.283060,0.019059,Rock,Rock


In [6]:
# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
#import graphviz

In [25]:
y = df_flat["genre_top_grouped"].dropna()
X = df_flat.drop(["genre_top","genre_top_grouped"],axis=1).loc[y.index]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

In [19]:
y_pred = rf.predict(X_test)

In [20]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6271169354838709


In [23]:
Counter(df_flat['genre_top'])

Counter({'Rock': 14182,
         'Experimental': 10608,
         'Electronic': 9372,
         'Hip-Hop': 3552,
         'Folk': 2803,
         'Pop': 2332,
         'Instrumental': 2079,
         'International': 1389,
         'Classical': 1230,
         'Jazz': 571,
         'Old-Time / Historic': 554,
         'Spoken': 423,
         'Country': 194,
         'Soul-RnB': 175,
         'Blues': 110,
         'Easy Listening': 24})